# Nov 30

In [5]:
import numpy as np
import pandas as pd
!cd .. && python ./scripts/download.py

Directory "data" already exists
File "data/modcloth_data.json.gz" already exists
File "data/renttherunway_data.json.gz" already exists
File "data/train_data.json" already exists


In [49]:
# Load datasets
train_df = pd.read_json('../data/train_data.json')
old_df = pd.read_json('../data/renttherunway_data.json.gz',
                      lines=True,
                      compression='gzip')
train_df.columns, old_df.columns

(Index(['item_name', 'price', 'review_summary', 'review', 'user_name', 'fit',
        'rented_for', 'usually_wear', 'size', 'age', 'height', 'bust_size',
        'weight', 'body_type', 'rating'],
       dtype='object'),
 Index(['fit', 'user_id', 'bust size', 'item_id', 'weight', 'rating',
        'rented for', 'review_text', 'body type', 'review_summary', 'category',
        'height', 'size', 'age', 'review_date'],
       dtype='object'))

## Convert the format of training data


In [50]:
# Rearrange columns

train_df = train_df[[
    'fit', 'item_name', 'size', 'price', 'user_name', 'rented_for',
    'usually_wear', 'age', 'height', 'weight', 'body_type', 'bust_size',
    'review_summary', 'review', 'rating'
]]
train_df.shape

(87766, 15)

### item_name

In [51]:
train_df['item_name'] = train_df['item_name'].astype('string', copy=False)
train_df['item_name'].value_counts()

MSGM
Blue Popover Ruffle Jumpsuit                 40
La Petite Robe di Chiara Boni
Rebecca Jumpsuit    40
M.i.h. Jeans
Plaid Ryley Coat                     40
Hunter Bell
Black Kelly Jumpsuit                  40
A.L.C.
Helena Sweater                             40
                                                  ..
Pravella Jumpsuit"                                 1
Paco Rabanne
Halter Crop Top                       1
Sandro
Noella Tweed Blazer                         1
MINKPINK
Kendra Knot Top                           1
﻿"Self-Portrait                                    1
Name: item_name, Length: 4460, dtype: Int64

In [52]:
# WTF?
wtf = train_df['item_name'].str.contains('\ufeff')
train_df[wtf]

,fit,item_name,size,price,user_name,rented_for,usually_wear,age,height,weight,body_type,bust_size,review_summary,review,rating
172,,"﻿""Ulla Johnson",,,,,,,,,,,,,
302,,"﻿""One Grey Day",,,,,,,,,,,,,
562,,"﻿""Baum und Pferdgarten",,,,,,,,,,,,,
628,,"﻿""Cedric Charlier",,,,,,,,,,,,,
853,,"﻿""Cedric Charlier",,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86443,,"﻿""Tory Burch",,,,,,,,,,,,,
86484,,"﻿""VEDA",,,,,,,,,,,,,
87075,,"﻿""Jason Wu Collective",,,,,,,,,,,,,
87314,,"﻿""Something Navy",,,,,,,,,,,,,


In [53]:
train_df = train_df[~wtf]

In [54]:
train_df['item_name'].value_counts()

Hutch
Cropped Zia Jumpsuit                 40
A.L.C.
Camden Sweater                      40
C/MEO COLLECTIVE
Yellow Crew Neck Top      40
STYLESTALKER
Rachelle Knit Top             40
Hutch
Betty Leaf Romper                    40
                                           ..
Saylor
Tosh Cropped Sweater                 1
Benjamin Benmoyal
Cassette Overshirt        1
Space Dye Sweater"                          1
NOIZE
Suren Jacket                          1
La DoubleJ
St. Moritz Reversible Jacket     1
Name: item_name, Length: 4325, dtype: Int64

In [67]:
new_cols = train_df['item_name'].str.split('\n', expand=True)
train_df.insert(1, 'Brand', new_cols[0])
train_df.insert(1, 'ItemName', new_cols[1])
train_df.drop(columns=['item_name'], inplace=True)
train_df.sort_values(by=['Brand', 'ItemName'], inplace=True)
train_df

,fit,ItemName,Brand,size,price,user_name,rented_for,usually_wear,age,height,weight,body_type,bust_size,review_summary,review,rating
976,,Chevron Turtleneck Sweater,(nude),NONE,$295,Kim,Other,10,28,"4' 8""",,,,Cute but...,"Such a cute sweater for fall, but it is soooo ...",3
1686,Large,Chevron Turtleneck Sweater,(nude),38,$295,Rochelle,Work,0,50,,108LBS,PETITE,32AA,Nice pattern,I got lots of compliments on this sweater! Roo...,5
2204,True to Size,Chevron Turtleneck Sweater,(nude),40,$295,Lia,Everyday,4,47,,130LBS,ATHLETIC,34B,"I love this sweater! It has a beautiful fit, w...",The sweater sort of drapes in a really feminin...,5
5085,Large,Chevron Turtleneck Sweater,(nude),None,$295,Georgie,Other,10,43,"5' 3""",140LBS,,,I didn't wear it!,It is adorable but you need to wear something ...,2
5786,,Chevron Turtleneck Sweater,(nude),46,$295,Amanda,Everyday,10,40,"5' 7""",165LBS,HOURGLASS,38B,Itchy but worth it,I ended up wearing this a few times. Perfect ...,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26922,Large,Lily Maternity Jumpsuit,soon maternity,XL,$120,Savannah,Everyday,10,28,,200LBS,HOURGLASS,36D,So comfy and slimming,I rented a smaller size (Large) and that fit p...,5
36515,True to Size,Lily Maternity Jumpsuit,soon maternity,S,$120,Jacqueline,Everyday,2,35,,125LBS,PETITE,34D,Love for third trimester,Great casual outfit for third trimester. Mater...,5
38920,True to Size,Lily Maternity Jumpsuit,soon maternity,S,$120,Emily,Everyday,4,39,"5' 4""",,,,So comfy,Perfect for late in my pregnancy,5
51062,Large,Lily Maternity Jumpsuit,soon maternity,NONE,$120,Rachel,,4,35,"5' 2""",123LBS,HOURGLASS,32C,Not cute,Pockets are wide!! As a petite person 20 weeks...,2


### fit

In [86]:
train_df['fit'] = train_df['fit'].astype('category', copy=False)
train_df.rename(columns={'fit': 'Fit'}, inplace=True)
train_df['Fit'].replace({'': np.nan}, inplace=True)
train_df['Fit'].value_counts()

True to Size    40881
Large           10162
Small            8784
Name: Fit, dtype: int64

### size

In [84]:
train_df['size'].unique()

array(['NONE', '38', '40', 'None', '46', '48', '44', '42', '6R', '8R',
       '2R', '4R', '10R', '12R', '0R', '10', '6', '12', '4', 'L', 'XS',
       'M', 'S', '0', '2', '8', nan, 'XL', 'XXL', '28R', '31R', '30R',
       '27R', '26R', '32R', '27', '28', '30', '24', '26', '29', '25',
       '31', '25R', '29R', '32', '18', '16', '14', '14R', '34', '36',
       'XSR', 'MR', 'LR', 'SR', 'XLR', 'M-L', 'P-S', '16R', '14L', '2P',
       '6P', '8L', 'XXS', '18R', '22R', '20R', '20', '22', '10L', '8P',
       'ML', 'SP', 'LL', 'XLL', 'SL', 'XSL', 'MP', '3X', '1X', '2X',
       'XSP', '1', '3', '38R', '42R', '-1', '2XL', '0P', '22WR', '4L',
       '16WL', '6L', '16WR', '18WR', '12P', '18WP', '20WR', '12L', '0L',
       '22W', '18W', '00', '1R', '3R', '00R', '5R', 'XXSR', '40R', '44R',
       '34R', '36R', '4P', '16L', '10P', '14P', '16P', '18L', 'XXLR',
       'XS-S', '2L', '50', '52', '54', 'ONESIZE', '18-20', '14-16',
       '22-24', 'S-M', 'L-XL', '48R', '46R', '52R', '50R', '5', '3XR',
     

In [87]:
train_df.to_csv('data/train_data_Nov_30.csv', index=False)

## Conclusion
- We arrange the columns to match the format on the website.
- We discard the corrupted data whose `item_name` starts with `\ufeff`.
- We 

## TODO: Convert the format of vanilla RentTheRunway dataset